### IMPORT LIBRARIES & DATA

In [1]:
import pandas as pd
import mysql.connector
import sqlalchemy as sa
import getpass

import numpy as np
import pickle
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

ENGLISH_WORDS = set(words.words())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import MinMaxScaler

In [4]:
from app.setup import setup
from models.tfidf import TfidfModel
pool = setup()

Current time:  2023-06-14 10:16:05


In [5]:
books_query = sa.text(
    "SELECT * FROM books;"
)
books_df = pd.read_sql_query(books_query, con=pool.connect())

In [6]:
ratings = pd.read_csv('./data/main/ratings_train.dat', sep=':', header=None)
ratings_test = pd.read_csv('./data/main/ratings_test.dat', sep=':', header=None)

### MODEL

In [13]:
data = books_df
id = 'goodreads_book_id'
docs = 'description'
model = TfidfModel(data, id, docs)
model.fit()

In [97]:
def get_user(id=1, scaler=None):
    profile = ratings.loc[ratings[0] == id]
    profile_test = ratings_test.loc[ratings_test[0] == id]
    x_train = profile[1].astype(str).tolist()
    y_train = profile[2].tolist()
    x_test = profile_test[1].astype(str).tolist()
    y_test = profile_test[2].tolist()
    if scaler:
        all_ratings = y_train + y_test
        scaler = scaler.fit(np.array(all_ratings).reshape(-1,1))
        y_train = scaler.transform(np.array(y_train).reshape(-1,1))
        y_test = scaler.transform(np.array(y_test).reshape(-1,1))
        y_train = y_train.flatten()
        y_test = y_test.flatten()
    return (x_train, y_train, x_test, y_test)

In [100]:
from sklearn.preprocessing import StandardScaler

In [108]:
x_train, y_train, x_test, y_test = get_user(9998, MinMaxScaler(feature_range=(0,1)))
model.update_user(x_train, y_train)
model.score(MSE, x_test, y_test)

0.05608765777404371